In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
import os
from os import listdir
from os.path import isfile, join
from itertools import combinations

In [ ]:
def generate_playlist_df(playlist_id, playlist_json):
    cols = ['plst_id', 'plst_name', 'track_id', 'track_name']
    data_col = []
    plst_name = playlist_json['name']
    
    for track in playlist_json['tracks']:
        track_id = ''.join(track['track_uri'].split(':')[2:])
        data_col.append([f'plst_{playlist_id}', plst_name, track_id, track['track_name']])
        
    plst_df = pd.DataFrame(data=data_col, columns=cols)
    return plst_df

def generate_file_df(start_index, file_json):
    dfs = []
    for i, plst_json in enumerate(file_json['playlists']):
        dfs.append(generate_playlist_df(start_index + i, plst_json))
        
    df_sum = pd.concat(dfs)
    return df_sum

def generate_spotify_df(data_path, start_index=None, end_index=None):
    assert (start_index is None and end_index is None) or \
        (start_index is not None and end_index is not None), 'Set both or none indices.'
    json_names = [f for f in listdir(data_path) if isfile(join(data_path, f)) and '.json' in f]
    
    num_playlists = start_index if start_index is not None else 0
    section = json_names if start_index is None else json_names[start_index:end_index]
    dfs = []
    
    for file_name in tqdm(section, desc='Files processed: ', unit='files', total=len(section)):
        with open(join(data_path, file_name)) as json_file:
            data = json.load(json_file)
            
        dfs.append(generate_file_df(num_playlists, data))
        num_playlists += len(data['playlists'])
        
    df_total = pd.concat(dfs)
    return df_total

data_path = '../data/' #change path
df = generate_spotify_df(data_path, 0, len(os.listdir()))